Búsqueda de palabras semanticamente similares y análisis de sentimiento

In [1]:
import numpy as np
import pandas as pd
import string

from sklearn.model_selection import train_test_split
from sklearn import metrics, neighbors

In [2]:
# funciones varias

def cargar_vocabulario(filename):
  lexicon = []
  with open(filename, encoding='latin-1') as infile:
      for line in infile:
          line = line.rstrip()
          if line and not line.startswith(';'):
              lexicon.append(line)
  return lexicon

def vectorizar_palabra(strng, alphabet=string.ascii_lowercase):
  vector = np.zeros(len(alphabet))
  for letter in strng:
    vector += np.array([0 if char != letter else 1 for char in alphabet])
  return vector

def normalizar_vector(vector):
  v = vector / np.sum(vector)
  return v

def lista_a_matriz(lista, vert = True):
  if vert:
    return np.vstack(lista)
  else:
    return np.hstack(lista)

In [3]:
# 1. construir/cargar vocabulario de palabras con sentimiento
# a) cargar los archivos del sitio en la carpeta data. usar los botones para subir archivos
# b) cargar en variables separadas los vocabularios positivos y negativos

vocabulario_pos = cargar_vocabulario('positive-words.txt')
vocabulario_neg = cargar_vocabulario('negative-words.txt')

In [4]:
# 2. para cada palabra, constuir un vector descriptor
# a) crear un array de 26 dimensiones por palabra, cada una correspondiente a la cantidad de cada letra
# b) normalizar cada array, para que la suma sea 1.
# c) cargar en variables separadas los vectores positivos y negativos

vectores_pos = lista_a_matriz([normalizar_vector(vectorizar_palabra(palabra)) for palabra in vocabulario_pos])
vectores_neg = lista_a_matriz([normalizar_vector(vectorizar_palabra(palabra)) for palabra in vocabulario_neg])

In [ ]:
# 3. obtener el sentimiento con knn y analizar algunas de las clasificaciones
# a) generar sets de entrenamiento y test (usar función train_test_split de sklearn)
# b) entrenar y predecir con neighbors.KNeighborsClassifier
# c) analizar rendimiento en base a metrics.confusion_matrix y metrics.accuracy_score

X = lista_a_matriz([vectores_pos, vectores_neg])
Y = lista_a_matriz([np.ones((len(vectores_pos))), -np.ones((len(vectores_neg)))], vert=False)
train_vectors, test_vectors, train_targets, test_targets = train_test_split(X, Y, test_size=0.1, random_state=0)

knn = neighbors.KNeighborsClassifier(n_neighbors=1)
knn.fit(train_vectors, train_targets)
predicted = knn.predict(test_vectors)
print(f'Confusion matrix:\n {metrics.confusion_matrix(test_targets, predicted)}')
print(f'Accuracy: {metrics.balanced_accuracy_score(test_targets, predicted)}')

In [ ]:
# 4. para algunas palabras, encontrar sus k-vecinos cercanos y analizar
# a) entrenar un encontrador de vecinos con neighbors.NearestNeighbors
# b) elegir palabras
# c) encontrar vecinos de palabras con función kneighbors
# d) imprimir información relevante del/los vecinos

vecinos = neighbors.NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)

voc = vocabulario_pos+vocabulario_neg
palabras = [0, 220, 1340, 2500]
for p in palabras:
  distancia, indice = vecinos.kneighbors(X[p].reshape(1,-1))
  print(f'Palabra = {voc[indice[0,0]]},\t Vecino = {voc[indice[0,1]]},\t \
  Sentimiento = {"Positivo" if indice[0,1] < len(vocabulario_pos) else "Negativo"},\t Distancia = {distancia[0,1]}')